# Develop a Web chatbot using gradio and OpenAI's GPT-4o-mini

In [1]:
import gradio as gr
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from openai import OpenAI

In [2]:
load_dotenv(override=True)
openai = OpenAI()

## Read pdf files and use them as context for a chatbot using gradio and OpenAI's GPT-4o-mini

In [5]:
reader = PdfReader("../resume/resume_khangbuitranduy_updated.pdf")
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

In [6]:
print(resume)

(+84)-815-594-665 |       khang.buitranduycse@hcmut.edu.vn |      khangbkk23 |       khangbkk23
KHANG BUI TRAN DUY
AI ENGINEER / DATA SCIENCE 
2023 - 2027
May 2025 - PresentI’m a highly motivated third-year Computer Science student with a strong foundation in Machine Learning, Data
Science, Computer Vision, and Natural Language Processing. I’m passionate about Artificial Intelligence,
continuously expanding my knowledge and applying it to real-world problems. I have hands-on experience with
TensorFlow, PyTorch, and Scikit-learn, along with solid skills in Machine Learning and Deep Learning, and growing
expertise in Reinforcement Learning. Beyond academics, I actively engage in social and community activities, which have
strengthened my teamwork, communication, and leadership skills. As an aspiring AI Engineer and Data Scientist, I am
eager to collaborate with experts, tackle challenging projects, and further develop my expertise with the goal of building
scalable, intelligent systems a

In [ ]:
name = "Duy Khang"

In [ ]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."